In [23]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/ukrsyllab.py /content/ukrsyllab.py
import ukrsyllab #syllabificator -- initially for russian(https://github.com/Koziev/rusyllab), slightly modified for ukrainain 
import pandas, re, random
!pip install git+https://github.com/Desklop/Uk_Stemmer
from uk_stemmer import UkStemmer #stemmer for ukrainian 
from ast import literal_eval 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Desklop/Uk_Stemmer to /tmp/pip-req-build-br933kht
  Running command git clone --filter=blob:none --quiet https://github.com/Desklop/Uk_Stemmer /tmp/pip-req-build-br933kht
  Resolved https://github.com/Desklop/Uk_Stemmer to commit a700ae1bd9b69ad84d311d089e8bc95ab7fab44d
  Preparing metadata (setup.py) ... done


In [24]:
file = files.upload() #uploading the dataset

Saving uk_UA.csv to uk_UA (1).csv


In [25]:
dataset = open("uk_UA.csv", "r", encoding="utf8") #opening and reading the dataset
dataset = dataset.read()
words = dataset.split("\n")

In [26]:
stems = []
for w in words[:100000]: 
  w = re.sub("\w'\w", "", w) #remove apostrophe words
  w = re.sub("\w-\w", "", w) #remove hyphenated words
  stemmer = UkStemmer()
  word = stemmer.stem_word(w) 
  stemmer = UkStemmer() #stem the words from the dataset
  if w in words and len(w) > 1 and w.lower() == w: 
        if w not in stems:
          stems.append(w) #append the stems list with unique lower-case two(or more)-syllable stems 

In [27]:
sound_prob = {}
in_m = {}
for word in stems:
  i = 0 #index value
  morphemes = ukrsyllab.split_words(word.split()) #get syllables -- bigrams used to form a matrix
  if len(morphemes) != 1: 
    if morphemes[0] not in in_m:
      in_m[morphemes[0]] = 1/(len(stems))
    else:
      in_m[morphemes[0]] += 1/(len(stems))
        #print(in_m)
  while i < (len(morphemes) - 1): #while the index is still in the bound of the word
      b = morphemes[i]
      n = morphemes[i+1]
      if n != 0: #in the cases where we have two biagrams following each other, (the initial biagram is not the last biagram of the word)
        if b in sound_prob: #if the first biagram is in the database of biagrams
          if n in sound_prob: #if the second one is in the database as well
            bInd = list(sound_prob).index(n) #find the index of the second biagram
            sound_prob[b][bInd] = int(sound_prob[b][bInd]) + 1 #and manipulate the probability value in the row of b that has the index of the following biagram
          else:
            probs = len(sound_prob) * "0" #if the second biagram does not exist in the dataset, we need to add it to the matrix and it needs a value row full of 0's (because its frequency with other sounds is 0)
            sound_prob[n] = list(probs) 
            for x in sound_prob: sound_prob[x].append(0) #every row of the matrix should be appended with 0 because the new sound that is added is also added as a new column that has never occured with the other sounds before
            sound_prob[b][-1] = 1 #the last entry of the probability of b should be altered to 1 because the new added sound appeared after that so +1 occurence
        else:
          probs = len(sound_prob) * "0" #if b does not exist in the dataset we should add it
          sound_prob[b] = list(probs)
          for x in sound_prob: sound_prob[x].append(0)
          if n in sound_prob:
            bInd = list(sound_prob).index(n)
            sound_prob[b][bInd] = int(sound_prob[b][bInd]) + 1
          else:
            sound_prob[n] = list(probs)
            sound_prob[n].append(0)
            for x in sound_prob: sound_prob[x].append(0)
            sound_prob[b][-1] = 1
      i += 1
df = pandas.DataFrame.from_dict(sound_prob, orient="index", columns=sound_prob.keys()) #display the matrix
df = df.applymap(int) #make every entry of the matrix an integer
df = df/len(sound_prob.keys()) #calculate the probabilities each occurence should be divided with all occurences
'''
with pandas.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ): print(df)
                       '''

"\nwith pandas.option_context('display.max_rows', None,\n                       'display.max_columns', None,\n                       'display.precision', 3,\n                       ): print(df)\n                       "

In [28]:
def suffixation (p_list): #ascribe suffixes to the pseudostems to make them morphologically recognizable 
  global p_words
  p_words = []
  suffixes = ['о-таки', 'о-то','но','цька','ий','ик','ник','івник','льник','иво','аль','ень','ець','ість','тель','иця','иня','ння','іння','ання','яння','ення','иння','еня','ечок','ечка','ечко','ичок','ичка','енко','енько','исько','ище','івка','овка','ок','ир','ист','изм','ір','іст','ізм','яти','ати','іти']
  ending = ['б', 'в', 'г', 'ґ', 'д', 'ж', 'з', 'к', 'л', 'м', 'н', 'п', 'р', 'с', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ']
  for instance in p_list:
      if (list(filter(instance.endswith, ending))): #if the ending of the pseudostem matches any ending of the 'ending' list, it gets a random suffix from the 'suffixes' list       
        norm = instance + random.choice(suffixes)
        if norm not in p_words: #pseudowords list gets appended with the unique pseudoword
          p_words.append(norm)
  return p_words

In [29]:
def cleaning (ps_list): #wipe out repeating syllables, vowels or consonants (normalize them to make more natural for ukrainian)
  ps_list = re.sub(r'(.+?)\1+', r'\1', str(ps_list))
  ps_list = literal_eval(str(ps_list)) #as the list needs to be converted into a string in the previous line, it gets converted back into a list
  return ps_list

In [30]:
pre_p_words = [] 
for j in range(20):
  pword = []
  i = 0
  plen = 4
  while i < plen:
    if pword == []:
      sound = random.choice(list(sound_prob.keys())) #start with a morpheme that excludes "й" or "ь" as first letters
      if sound[0] == "й" or sound[0] == "ь":
        sound = random.choice(list(sound_prob.keys())) 
      i += len(sound)
    else:
      sound = random.choices(list(sound_prob.keys()), list(df[pword[-1]]))[0] #proceed with other morphemes until the length requirement is satisfied
      i += len(sound)
    pword.append(sound)
  pword = "".join(pword)
  pre_p_words.append(pword)
  p_words = suffixation(pre_p_words) #run function to suffixate the stems
  p_words = cleaning(p_words) #run function to normalize the pseudowords 
print(p_words)

['вищрізок', 'чемстець', 'літровка', 'вмавегльник', 'одшвегець', 'цадверяня', 'ландсичок', 'фізіненя', 'вічодіти', 'вістізм']


In [34]:
def pairs(dataset):
  uk_pos = {"SUBJECT": [],
            "PREDICATE": [],
            "ATTRIBUTE":[],
            "OBJECT": [],
            "ADVERBIAL MODIFIER": []}

  subj_suff = ['ик','ник','івник','льник','иво','аль','ень','ець','ість','тель','иця','иня','ння','іння','ання','яння','ення','иння','еня','ечок','ечка','ечко','ичок','ичка','енко','исько','ище','івка','овка','ок','ир','ист','изм','ір','іст','ізм']
  pred_suff = ['ти']
  attr_suff = ['ий']
  mod_suff = ['-таки', '-то', 'но', 'ицька' ]

  pr = []
  attr = []
  obj = []

  for word in dataset:
      if any(word.endswith(s) for s in subj_suff): #adding pseudowords in the nominative case to the dictionary
            uk_pos["SUBJECT"].append(word)
            obj.append(word)
      elif any(word.endswith(mod) for mod in mod_suff):
            uk_pos["ADVERBIAL MODIFIER"].append(word)
      elif any(word.endswith(p) for p in pred_suff): #adding pseudowords (later to be inflected (accusative)) to the respective lists
            pr.append(word)
      elif any(word.endswith(at) for at in attr_suff):
            attr.append(word)

  def infl_dict(infl_suff, inf_suff, lst, dict_key): #function that inflects predicates and attributes, and adds them to the dictionary
    for st in lst:
      for af in infl_suff:
        ret = re.sub(inf_suff, af, st) 
        uk_pos[dict_key].append(ret)


  infl_dict(infl_suff=['в','ла', 'ло'], inf_suff = 'ти', lst=pr, dict_key="PREDICATE")
  infl_dict(infl_suff=['ої','ого'], inf_suff = 'ий', lst=attr, dict_key="ATTRIBUTE")

  def infl_dict_obj (gendered_suffix, inflection):
      for ob in obj:
        if any(ob.endswith(s) for s in gendered_suffix):
          if ob[-1] in ['м', 'р', 'к', 'т']:
            ob = ob + inflection
          elif ob[-1] in ['ь']:
            ob = ob[:-1] + inflection
          elif ob[-1] in ['я']:
            ob = ob[:-1] + inflection
          elif ob[-1] in ['а']:
            ob = ob[:-1] + inflection
          elif ob[-1] in ['о', 'е']:
            ob = ob[:-1] + inflection
          uk_pos["OBJECT"].append(ob)

  infl_dict_obj(gendered_suffix=['ик','ник','івник','льник','ок','ир','ист','изм', 'ір', 'іст', 'ізм'], inflection = 'а') #masculine nouns that end in a hard consonant
  infl_dict_obj(gendered_suffix= ['аль', 'ень' 'тель', 'ець'], inflection = 'я') #masculine nouns that end in a soft consonant
  infl_dict_obj(gendered_suffix=['иця','иня','ння','іння','ання','яння','иння'], inflection = 'і') #feminine nouns that end in a soft consonant
  infl_dict_obj(gendered_suffix=['ичка', 'івка', 'овка'], inflection = 'и') #feminine nouns that end in a hard consonant
  infl_dict_obj(gendered_suffix=['иво', 'ечко', 'енко', 'исько','ище'], inflection = 'а') #neutral nouns

  '''
  the neutral word order in ukrainian is SVO. however, other orders are possible due to inflections. here are some of the most common.
  '''

  def sent_generator(dic):
    print(dic)
    try:
      subj = random.choice(list(dic["SUBJECT"]))
    except:
      subj = ""
    try:
      pred = random.choice(list(dic["PREDICATE"]))
    except:
      pred = ""
    try:
      attr = random.choice(list(dic["ATTRIBUTE"]))
    except:
      attr = ""
    try:
      obje = random.choice(list(dic["OBJECT"]))
    except:
      obje = ""
    try:
      adve = random.choice(list(dic["ADVERBIAL MODIFIER"]))
    except:
      adve = ""

    if re.search('[ое]$', subj): # coordinating subject and predicate by gender
        while not re.search('ло$', pred):
          pred = random.choice(list(dic["PREDICATE"]))     
    elif re.search('[аяь]$', subj):
        while not re.search('ла$', pred):
          pred = random.choice(list(dic["PREDICATE"]))
    elif re.search('[кмстр]$', subj):
        while not re.search('в$', pred):
          pred = random.choice(list(dic["PREDICATE"]))
    
    if re.search('[иі]$', obje): # coordinating attribute and object by gender
        while attr and not re.search('ої$', attr):
          attr = random.choice(list(dic["ATTRIBUTE"]))
    elif re.search('[ая]$', obje): 
        while attr and not re.search('ого$', attr):
          attr = random.choice(list(dic["ATTRIBUTE"]))     

    sent_str = random.choice(["sent_str1", "sent_str2",  "sent_str3"])
    if sent_str == "sent_str1":
      sent = subj + " " + pred + " " + attr + " " + obje + " " + adve + "."
    elif sent_str == "sent_str1":
      sent = adve + " " + subj + " " + pred + " " + attr + " " + obj + "."
    else: 
      sent = attr + " " + obje + " " + pred + " " + subj + " " + adve + "."
    sent = sent.capitalize()
    return sent

  for i in range(10):
    print(sent_generator(uk_pos))

pairs(literal_eval(str(p_words)))


{'SUBJECT': ['вищрізок', 'чемстець', 'літровка', 'вмавегльник', 'одшвегець', 'ландсичок', 'фізіненя', 'вістізм'], 'PREDICATE': ['вічодів', 'вічоділа', 'вічоділо'], 'ATTRIBUTE': [], 'OBJECT': ['вищрізока', 'вмавегльника', 'ландсичока', 'вістізма', 'чемстеця', 'одшвегеця', 'літровки'], 'ADVERBIAL MODIFIER': []}
 вмавегльника вічодів вмавегльник .
{'SUBJECT': ['вищрізок', 'чемстець', 'літровка', 'вмавегльник', 'одшвегець', 'ландсичок', 'фізіненя', 'вістізм'], 'PREDICATE': ['вічодів', 'вічоділа', 'вічоділо'], 'ATTRIBUTE': [], 'OBJECT': ['вищрізока', 'вмавегльника', 'ландсичока', 'вістізма', 'чемстеця', 'одшвегеця', 'літровки'], 'ADVERBIAL MODIFIER': []}
 вмавегльника вічоділа одшвегець .
{'SUBJECT': ['вищрізок', 'чемстець', 'літровка', 'вмавегльник', 'одшвегець', 'ландсичок', 'фізіненя', 'вістізм'], 'PREDICATE': ['вічодів', 'вічоділа', 'вічоділо'], 'ATTRIBUTE': [], 'OBJECT': ['вищрізока', 'вмавегльника', 'ландсичока', 'вістізма', 'чемстеця', 'одшвегеця', 'літровки'], 'ADVERBIAL MODIFIER': 